# Multi Flying Circle

In [5]:
import pygame
import random as r

class Circle:
    """
    Circle class
    """
    def __init__(self, radius, x, y, color, vx, vy) -> None:
        self.radius = radius
        self.x = x
        self.y = y
        self.color = color
        self.vx = vx
        self.vy = vy

    def draw(self, screen) -> None:
        """
        Draw circle at (x, y)
        """
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)

    def move(self, window_width, window_height) -> None:
        """
        Move the circle by shift_x and shift_y
        """
        if self.x > window_width - self.radius or self.x < self.radius:
            self.vx = -self.vx
        if self.y > window_height - self.radius or self.y < self.radius:
            self.vy = -self.vy

        self.x += self.vx
        self.y += self.vy


if __name__ == "__main__":
    window_width = 500
    window_height = 500

    N = 20

    circles = []
    for i in range(N):
        radius = r.randint(10, 40)
        x = r.randint(radius, window_width-radius)
        y = r.randint(radius, window_height-radius)
        vx = r.randint(-5, 5)
        vy = r.randint(-5, 5)
        R = r.randint(0, 255)
        G = r.randint(0, 255)
        B = r.randint(0, 255)
        circles.append(Circle(radius, x, y, (R,G,B), vx, vy))

    pygame.init()

    screen = pygame.display.set_mode((window_width, window_height)) # set screen size
    pygame.display.set_caption("Multiple Flying Circle") # set the title
    clock = pygame.time.Clock() # Clock, I guess.

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                break

        screen.fill((255, 255, 255)) # fill with white

        for circle in circles:
            circle.move(window_width, window_height)
            circle.draw(screen)

        pygame.display.flip()

        clock.tick(60) # set to 144 fps

    pygame.quit()

# Collision of Multi Flying Circle

In [3]:
import pygame
import random as r

class Circle:
    """
    Circle class
    """
    flag = False
    def __init__(self, radius, x, y, color, vx, vy) -> None:
        self.radius = radius
        self.x = x
        self.y = y
        self.color = color
        self.vx = vx
        self.vy = vy

    def draw(self, screen) -> None:
        """
        Draw circle at (x, y)
        """
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)

    def move(self, window_width, window_height) -> None:
        """
        Move the circle by shift_x and shift_y
        """
        if self.x > window_width - self.radius or self.x < self.radius:
            self.vx = -self.vx
        if self.y > window_height - self.radius or self.y < self.radius:
            self.vy = -self.vy

        self.x += self.vx
        self.y += self.vy

    def collision(self, other) -> None:
        """
        Handle the collision between two circles
        """
        distance_center = ((self.x - other.x)**2 + (self.y - other.y)**2)**0.5

        if distance_center < self.radius + other.radius:
            nx = (other.x - self.x) / distance_center
            ny = (other.y - self.y) / distance_center

            # Calculate the relative velocity in the normal direction
            dvx = other.vx - self.vx
            dvy = other.vy - self.vy
            dot = dvx * nx + dvy * ny

            # Only process if circles are moving toward each other
            if dot > 0:
                return

            # Exchange velocities along the collision vector
            self.vx += dot * nx
            self.vy += dot * ny
            other.vx -= dot * nx
            other.vy -= dot * ny
    
if __name__ == "__main__":
    window_width = 500
    window_height = 500

    N = 20

    circles = []
    for i in range(N):
        radius = r.randint(10, 40)
        x = r.randint(radius, window_width-radius)
        y = r.randint(radius, window_height-radius)
        vx = r.randint(-5, 5)
        vy = r.randint(-5, 5)
        R = r.randint(0, 255)
        G = r.randint(0, 255)
        B = r.randint(0, 255)
        circles.append(Circle(radius, x, y, (R,G,B), vx, vy))

    pygame.init()

    screen = pygame.display.set_mode((window_width, window_height)) # set screen size
    pygame.display.set_caption("Multiple Collision between Flying Circle") # set the title
    clock = pygame.time.Clock() # Clock, I guess.

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                break

        screen.fill((255, 255, 255)) # fill with white

        for i in range(len(circles)):
            for j in range(i+1,len(circles)):
                if i == j:
                    continue
                circles[i].collision(circles[j])
            circles[i].move(window_width, window_height)
            circles[i].draw(screen)

        pygame.display.flip()

        clock.tick(60) # set to 60 fps

    pygame.quit()

# SOCKET

### Sending data (Client)

In [ ]:
import socket

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 4646        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    # s.sendall(b'GET / HTTP/1.1\r\nHost: test.net\r\n\r\n')
    s.send(b'Hi from client')
    data = s.recv(1024)
    print('Received', repr(data))


### Recieving data (Server)

In [26]:
import socket
import sys

HOST = None
PORT = 4646
s = None

for res in socket.getaddrinfo(HOST, PORT, socket.AF_UNSPEC,
                              socket.SOCK_STREAM, 0, socket.AI_PASSIVE):
    af, socktype, proto, canonname, sa = res
    try:
        s = socket.socket(af, socktype, proto)
    except OSError as msg:
        s = None
        continue

    try:
        s.bind(sa)
        s.listen(1)
    except OSError as msg:
        print("Error waiting for connection!")
        s.close()
        s = None
        continue
    break

print("server is listenning")
if s is None:
    print('could not open socket')
    sys.exit(1)

conn, addr = s.accept()
with conn:
    print('Connected by', addr)
    data = conn.recv(1024)
    print('Recieved from client: ',repr(data))

    # answer = b"HTTP/1.1 200 OK\r\nContent-Length: 14\r\nContent-Type: text/html\r\nHi Browser\r\n\r\n"
    conn.send(b'Hello from server!')

print("Server Finished")